In [29]:
WITH DeduplicatedData AS (
    SELECT 
        CustomerID,
        MonthlyCharges,
        Round(TenureMonths,0) as TenureMonths ,
        Gender,
        LastInteraction,
        ContractType,
        FavoriteColor,
        ROW_NUMBER() OVER (
            PARTITION BY CustomerID 
            ORDER BY LastInteraction DESC
        ) AS RowNum
    FROM 
        telco_data_quality_issues.csv
	)
SELECT 
    CASE WHEN TenureMonths IS NULL THEN 
	ROUND((SELECT AVG(TenureMonths) FROM telco_data_quality_issues.csv),0)
	ELSE TenureMonths END AS TenureMonths,
    CASE 
        WHEN Gender IN ('FEMALE', 'F', 'Female') THEN 'Female'
        WHEN Gender IN ('MALE', 'M', 'Male') THEN 'Male'
        ELSE Gender
    END AS Gender,
    LastInteraction,
    ContractType,
    COALESCE(NULLIF(FavoriteColor, ''), 'Unknown') AS FavoriteColor,
    CASE WHEN MonthlyCharges = -999 THEN 
	ROUND((SELECT AVG(MonthlyCharges) FROM telco_data_quality_issues.csv WHERE MonthlyCharges != -999),0)
	ELSE round(MonthlyCharges,2) END AS MonthlyCharges,
     CustomerID
FROM 
    DeduplicatedData
WHERE RowNum = 1;

,TenureMonths,Gender,LastInteraction,ContractType,FavoriteColor,MonthlyCharges,CustomerID
0,15.0,Female,2015-01-04 00:00:00+00:00,Month-to-Month,Blue,63.05,15
1,49.0,Female,2013-09-25 00:00:00+00:00,Two Year,Yellow,74.90,22
2,35.0,Male,2011-05-14 00:00:00+00:00,Two Year,Yellow,60.65,32
3,60.0,Male,2012-01-15 00:00:00+00:00,Invalid,Yellow,77.34,52
4,16.0,Female,2012-04-02 00:00:00+00:00,Invalid,Red,73.12,81
...,...,...,...,...,...,...,...
594,2.0,Female,2010-09-19 00:00:00+00:00,Invalid,Red,95.15,757
595,16.0,Male,2012-09-03 00:00:00+00:00,Month-to-Month,Blue,84.53,761
596,31.0,Male,2013-07-27 00:00:00+00:00,Invalid,Green,60.32,792
597,30.0,Male,2014-05-29 00:00:00+00:00,Month-to-Month,Green,70.00,831
